In [11]:
import pandas as pd
import torch
import nltk
import re
import json
from typing import List

try:
    nltk.data.find('tokenizers/punkt')
except LookupError:
    nltk.download('punkt')

In [ ]:
df = pd.read_csv('./data/train.tsv', sep='\t')

def tokenize_text(text: str) -> List[str]:
    text = re.sub(r'([a-z])- ([a-z])', '\1\2', text) # Saw hyphens like this in the middle of a lot of words e.g. "transmission elec- tron microscopy". Seem like line breaks
    return nltk.tokenize.word_tokenize(text)

pd.DataFrame([(tokenize_text(t),l) for t,l in df.to_records(index=False)], columns=df.columns)

In [ ]:
class TextDataset(torch.Dataset):
    def __init__(self, train=True):
        path = f'./data/{"train" if train else "test"}.tsv'
        self.data = pd.read_csv(path)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text = tokenize_text(self.data.iloc[idx, 0])
        label = self.data.iloc[idx, 1]
        return text, label

In [16]:
document = ''
with open('../data/raw/new_sentences_acs.txt') as f:
    for el in f.readlines():
        document += el.replace('\n', ' ')

sents = nltk.sent_tokenize(document)
with open('../data/new_sentences_acs.json', 'w') as f:
    json.dump(sents, f, indent=4)